In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

In [2]:
depression_df = pd.read_csv('datasets_scraped/depression-dataset.csv', index_col=0)
suic_df = pd.read_csv('datasets_scraped/suicide-dataset.csv', index_col=0)
depressed_df = pd.read_csv('datasets_scraped/depressed-dataset.csv', index_col=0)
sad_df = pd.read_csv('datasets_scraped/sad-dataset.csv', index_col=0)
happy_df = pd.read_csv('datasets_scraped/happy-dataset.csv', index_col=0)
hate_df = pd.read_csv('datasets_scraped/hate-dataset.csv', index_col=0)
thanksgiving_df = pd.read_csv('datasets_scraped/thanksgiving-dataset.csv', index_col=0)
life_df = pd.read_csv('datasets_scraped/life-dataset.csv', index_col=0)

kaggle_df = pd.read_csv('datasets_scraped/kaggle-dataset', index_col=0)

pd.set_option('display.max_colwidth', None)

In [3]:
datasets = [depression_df, suic_df, depressed_df, sad_df, happy_df, hate_df, thanksgiving_df, life_df,kaggle_df] 


df = pd.concat(datasets, ignore_index=True)
df.reset_index(drop=True, inplace=True)

In [4]:
df[df['text'].isnull()]

,username,description,text,sentiment
9117,poojou,-,NaN,NaN
9118,@Qantas @Qantas this is what's happening #sad #foreverloading https://t.co/1RvOMFkwJd,-5.0,NaN,NaN
20822,NaN,NaN,NaN,NaN
20823,7.166666666666667,NaN,NaN,NaN


In [5]:
df.drop(df.index[[9117, 9118, 20822, 20823]], inplace=True)
df

,username,description,text,sentiment
0,cjoanward_,WNAA Air Ambulance Fundraising Volunteer 🚁\nBusiness Owner 👩‍💻📱\nMental Health Advocate 👏\nLGBT 🏳️‍🌈,🌐 NEW ON THE WEBSITE 🌐\nThis week we are talking about #depression\nKnow anyone who suffers with depression?? Suffer with depression yourself??\nCheck out: https://t.co/M8qzkc4g4h and have a look at both issues :),-2.318182
1,hashtagJJSCORP,thoughts on thoughts on thoughts on art. #MindHacking sEnse @21stcentury #stayingweirdfory’all,They just did it understand #depression 🫖🥚⭐️✨❔💝,0.000000
2,zatecia_937,NaN,Sometimes I be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #ADHD #depression #FamilyGuy #fun,-1.666667
3,Serenityspeaki1,"Qualified integrative/EMDR counsellor specialising in mental health,trauma,bereavement,anxiety,depression based in Farnham Surrey","Let’s get talking this week, Mental Health Awareness week. Talking to a friend, colleague, parent, sibling or therapist can help. https://t.co/ATM0h2mzrl. #mentalhealthawareness #talking #anxiety #depression #trauma #yourjourneystartshere #therapy #counsellor #help #surrey https://t.co/wh2yxkXS1S",-1.000000
4,Imheret45140132,"A t-bot created to respond to @Iamdepr47974144, to demonstrate how to help someone who is depressed. This is not a replacement for actual help.",@Iamdepr47974144 How can I best support you right now? #mentalhealth #depression,6.428571
...,...,...,...,...
21214,WhenDeadSpeak,"Paranormal Kitty's book page. Sharing my love for the art and science of the paranormal. EVPs, events, Reiki healing & more!",Read about how a tiny Robin showed Susie her Mom was near #Life After Death https://t.co/HlKW72VuBZ https://t.co/GP6HKbNjrG,0.500000
21215,gmc004,Ggg :\n\nAdams' Carrot : The Missing Link : Hearts and Minds : Passing through his Light ¦| Enlightenment |¦ Nec Tamen Consumebatur || Progress or falter,"The banks have understood the error in their ways and fully understand without truth and honesty at the bedrock and foundation of family #Life we can never be secure, until we reach retirement!? #Ggg2021 #Godbless",4.000000
21216,artdealership,#contemporary #artcontemporain #digitalart #artcurator #portrait #visualart #acrylicpainting #illustration #love #streetart #mixedmedia #creative #artcollectors,#ArtificialIntelligence #AI [Things I can only say in #Blogs] #Poetry #Life #AR #VR #Creators #TimeTravel #GeneticMemory #Christ #amWriting #Love #Eternity https://t.co/Wc3DG5yoml,2.500000
21217,DisruptiveArte,#contemporary #artcontemporain #digitalart #artcurator #portrait #visualart #acrylicpainting #illustration #love #streetart #mixedmedia #creative #artcollectors,#ArtificialIntelligence #AI [Things I can only say in #Blogs] #Poetry #Life #AR #VR #Creators #TimeTravel #GeneticMemory #Christ #amWriting #Love #Eternity https://t.co/5BG6IV7MgI,2.500000


In [6]:
df.drop(['description'], axis=1, inplace=True)

In [7]:
df['text_lower']  = df['text'].str.lower()
df['text_lower'] = df['text_lower'].apply(lambda x: x.replace('\n', ' '))
df['text_lower'] = [line.replace('&amp;','and') for line in df['text_lower']]

In [8]:
df[df['text_lower'].isna()]

,username,text,sentiment,text_lower


In [9]:
def remove_usernames(text):
    user_pattern = re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-\.]))@([A-Za-z_]+[A-Za-z0-9_]+)')
    return user_pattern.sub(r'USER', text)

df['text_no_user'] = df['text_lower'].apply(remove_usernames)

In [10]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for converting emojis into word
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, " ".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text

# Function for converting emoticons into word
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', " ".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

df['text_no_emoji'] = df['text_no_user'].apply(convert_emoticons)
df['text_no_emoji'] = df['text_no_user'].apply(convert_emojis)

In [11]:
df['text_no_emoji'] = df['text_no_emoji'].apply(lambda x: x.replace('_', ' '))

In [12]:
# Function for url s
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'URL', text)

#Passing the function to 'text_rare'
df['text_no_url'] = df['text_no_emoji'].apply(remove_urls)

In [13]:
import contractions

def word_expand(text):
    expanded_words = []       
    for word in text.split():
        expanded_words.append(contractions.fix(word))
    expanded_text = ' '.join(expanded_words)
    return expanded_text
    
df['text_expanded'] = df['text_no_url'].apply(word_expand)
df.head()

,username,text,sentiment,text_lower,text_no_user,text_no_emoji,text_no_url,text_expanded
0,cjoanward_,🌐 NEW ON THE WEBSITE 🌐\nThis week we are talking about #depression\nKnow anyone who suffers with depression?? Suffer with depression yourself??\nCheck out: https://t.co/M8qzkc4g4h and have a look at both issues :),-2.318182,🌐 new on the website 🌐 this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),🌐 new on the website 🌐 this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: URL and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: URL and have a look at both issues :)
1,hashtagJJSCORP,They just did it understand #depression 🫖🥚⭐️✨❔💝,0.000000,they just did it understand #depression 🫖🥚⭐️✨❔💝,they just did it understand #depression 🫖🥚⭐️✨❔💝,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon
2,zatecia_937,Sometimes I be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #ADHD #depression #FamilyGuy #fun,-1.666667,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #adhd #depression #familyguy #fun,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #adhd #depression #familyguy #fun,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you loudly crying faceface with tears of joyface with tears of joy #bipolar #adhd #depression #familyguy #fun,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you loudly crying faceface with tears of joyface with tears of joy #bipolar #adhd #depression #familyguy #fun,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you are depressed about the scenarios you make in your head” nothing ever happens to you loudly crying faceface with tears of joyface with tears of joy #bipolar #adhd #depression #familyguy #fun
3,Serenityspeaki1,"Let’s get talking this week, Mental Health Awareness week. Talking to a friend, colleague, parent, sibling or therapist can help. https://t.co/ATM0h2mzrl. #mentalhealthawareness #talking #anxiety #depression #trauma #yourjourneystartshere #therapy #counsellor #help #surrey https://t.co/wh2yxkXS1S",-1.000000,"let’s get talking this week, mental health awareness week. talking to a friend, colleague, parent, sibling or therapist can help. https://t.co/atm0h2mzrl. #menta

In [14]:
# # Spell check using text blob 
# # too long

# from textblob import TextBlob
# df['text_no_url'].apply(lambda x: str(TextBlob(x).correct()))

In [15]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV} # Pos tag, used Noun, Verb, Adjective and Adverb
# Function for lemmatization using POS tag
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

# df["text_lemma"] = df["text_expanded"].apply(lemmatize_words)
df["text_lemma"] = df["text_expanded"].apply(lemmatize_words)


In [16]:
def remove_hashtags(text):
    no_hash = re.compile("#(\w+)")
    return no_hash.sub(r' ', text)

df['text_no_hash'] = df['text_lemma'].apply(remove_hashtags)

In [17]:
#Creating function for tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['text_tokenized'] = df['text_no_hash'].apply(lambda x: tokenization(x.lower()))

In [18]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

df['processed_data'] = [TreebankWordDetokenizer().detokenize(text) for text in df['text_tokenized']]

In [19]:
from textblob import TextBlob
sentiment = []

for sentence in df['processed_data']: 
    sentiment.append(TextBlob(sentence).sentiment.polarity*10)

In [20]:
df['sentiment_processed'] = sentiment

In [21]:
textneg_filtered = df[df['sentiment_processed'] < -5]

In [22]:
df

,username,text,sentiment,text_lower,text_no_user,text_no_emoji,text_no_url,text_expanded,text_lemma,text_no_hash,text_tokenized,processed_data,sentiment_processed
0,cjoanward_,🌐 NEW ON THE WEBSITE 🌐\nThis week we are talking about #depression\nKnow anyone who suffers with depression?? Suffer with depression yourself??\nCheck out: https://t.co/M8qzkc4g4h and have a look at both issues :),-2.318182,🌐 new on the website 🌐 this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),🌐 new on the website 🌐 this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: URL and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: URL and have a look at both issues :),globe with meridian new on the website globe with meridian this week we be talk about #depression know anyone who suffer with depression?? suffer with depression yourself?? check out: URL and have a look at both issue :),globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression?? suffer with depression yourself?? check out: URL and have a look at both issue :),"[globe, with, meridian, new, on, the, website, globe, with, meridian, this, week, we, be, talk, about, know, anyone, who, suffer, with, depression, suffer, with, depression, yourself, check, out, url, and, have, a, look, at, both, issue, ]",globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression suffer with depression yourself check out url and have a look at both issue,1.363636
1,hashtagJJSCORP,They just did it understand #depression 🫖🥚⭐️✨❔💝,0.000000,they just did it understand #depression 🫖🥚⭐️✨❔💝,they just did it understand #depression 🫖🥚⭐️✨❔💝,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just do it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just do it understand 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,"[they, just, do, it, understand, eggwhite, medium, star, sparkleswhite, question, markheart, with, ribbon]",they just do it understand eggwhite medium star sparkleswhite question markheart with ribbon,0.000000
2,zatecia_937,Sometimes I be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #ADHD #depression #FamilyGuy #fun,-1.666667,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #adhd #depression #familyguy #fun,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #adhd #depressio

In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiment_vader = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    return sentiment_vader.polarity_scores(text)['compound']*10


df['new_vader_sentiment'] = df['processed_data'].apply(vader_sentiment)

In [24]:
df

,username,text,sentiment,text_lower,text_no_user,text_no_emoji,text_no_url,text_expanded,text_lemma,text_no_hash,text_tokenized,processed_data,sentiment_processed,new_vader_sentiment
0,cjoanward_,🌐 NEW ON THE WEBSITE 🌐\nThis week we are talking about #depression\nKnow anyone who suffers with depression?? Suffer with depression yourself??\nCheck out: https://t.co/M8qzkc4g4h and have a look at both issues :),-2.318182,🌐 new on the website 🌐 this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),🌐 new on the website 🌐 this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: URL and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: URL and have a look at both issues :),globe with meridian new on the website globe with meridian this week we be talk about #depression know anyone who suffer with depression?? suffer with depression yourself?? check out: URL and have a look at both issue :),globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression?? suffer with depression yourself?? check out: URL and have a look at both issue :),"[globe, with, meridian, new, on, the, website, globe, with, meridian, this, week, we, be, talk, about, know, anyone, who, suffer, with, depression, suffer, with, depression, yourself, check, out, url, and, have, a, look, at, both, issue, ]",globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression suffer with depression yourself check out url and have a look at both issue,1.363636,-9.371
1,hashtagJJSCORP,They just did it understand #depression 🫖🥚⭐️✨❔💝,0.000000,they just did it understand #depression 🫖🥚⭐️✨❔💝,they just did it understand #depression 🫖🥚⭐️✨❔💝,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just do it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just do it understand 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,"[they, just, do, it, understand, eggwhite, medium, star, sparkleswhite, question, markheart, with, ribbon]",they just do it understand eggwhite medium star sparkleswhite question markheart with ribbon,0.000000,0.000
2,zatecia_937,Sometimes I be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #ADHD #depression #FamilyGuy #fun,-1.666667,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #adhd #depression #familyguy #fun,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to 

In [25]:
def sent_to_text(sentiment, sentiment2):
    if sentiment < 0 and sentiment2 < 0:
        sentiment = 'neg'
    elif sentiment >= 0 or sentiment2 >= 0:
        sentiment = 'pos'

    return sentiment

# df['label'] = df[['sentiment_processed', 'new_vader_sentiment']].apply(sent_to_text)
df['label'] = df.apply(lambda x: sent_to_text(x.sentiment_processed, x.new_vader_sentiment), axis=1)

df

,username,text,sentiment,text_lower,text_no_user,text_no_emoji,text_no_url,text_expanded,text_lemma,text_no_hash,text_tokenized,processed_data,sentiment_processed,new_vader_sentiment,label
0,cjoanward_,🌐 NEW ON THE WEBSITE 🌐\nThis week we are talking about #depression\nKnow anyone who suffers with depression?? Suffer with depression yourself??\nCheck out: https://t.co/M8qzkc4g4h and have a look at both issues :),-2.318182,🌐 new on the website 🌐 this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),🌐 new on the website 🌐 this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: https://t.co/m8qzkc4g4h and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: URL and have a look at both issues :),globe with meridians new on the website globe with meridians this week we are talking about #depression know anyone who suffers with depression?? suffer with depression yourself?? check out: URL and have a look at both issues :),globe with meridian new on the website globe with meridian this week we be talk about #depression know anyone who suffer with depression?? suffer with depression yourself?? check out: URL and have a look at both issue :),globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression?? suffer with depression yourself?? check out: URL and have a look at both issue :),"[globe, with, meridian, new, on, the, website, globe, with, meridian, this, week, we, be, talk, about, know, anyone, who, suffer, with, depression, suffer, with, depression, yourself, check, out, url, and, have, a, look, at, both, issue, ]",globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression suffer with depression yourself check out url and have a look at both issue,1.363636,-9.371,pos
1,hashtagJJSCORP,They just did it understand #depression 🫖🥚⭐️✨❔💝,0.000000,they just did it understand #depression 🫖🥚⭐️✨❔💝,they just did it understand #depression 🫖🥚⭐️✨❔💝,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just did it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just do it understand #depression 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,they just do it understand 🫖eggwhite medium star️sparkleswhite question markheart with ribbon,"[they, just, do, it, understand, eggwhite, medium, star, sparkleswhite, question, markheart, with, ribbon]",they just do it understand eggwhite medium star sparkleswhite question markheart with ribbon,0.000000,0.000,pos
2,zatecia_937,Sometimes I be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #ADHD #depression #FamilyGuy #fun,-1.666667,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #adhd #depression #familyguy #fun,sometimes i be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ev

In [26]:
final_df = df.drop(['username', 'text', 'sentiment', 'text_lower', 'text_no_user', 'text_no_url', 'text_no_emoji', 'text_expanded', 'text_lemma', 'text_no_hash', 'text_tokenized', 'sentiment_processed', 'new_vader_sentiment'], axis=1)

In [27]:
final_df

,processed_data,label
0,globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression suffer with depression yourself check out url and have a look at both issue,pos
1,they just do it understand eggwhite medium star sparkleswhite question markheart with ribbon,pos
2,sometimes i be think to myself when some of you post depress shit like what the hell you be depress about the scenario you make in your head nothing ever happen to you loudly cry faceface with tear of joyface with tear of joy,pos
3,let u get talk this week mental health awareness week talk to a friend colleague parent sibling or therapist can help url url,pos
4,user how can i best support you right now,pos
...,...,...
21214,read about how a tiny robin show susie her mom be near after death url url,pos
21215,the bank have understand the error in their way and fully understand without truth and honesty at the bedrock and foundation of family we can never be secure until we reach retirement,pos
21216,things i can only say in url,pos
21217,things i can only say in url,pos


In [28]:
final_df.rename(columns = {'processed_data':'tweet'}, inplace = True)
final_df
# final_df['label'] = df.label.map({'pos':2, 'neg':1})

,tweet,label
0,globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression suffer with depression yourself check out url and have a look at both issue,pos
1,they just do it understand eggwhite medium star sparkleswhite question markheart with ribbon,pos
2,sometimes i be think to myself when some of you post depress shit like what the hell you be depress about the scenario you make in your head nothing ever happen to you loudly cry faceface with tear of joyface with tear of joy,pos
3,let u get talk this week mental health awareness week talk to a friend colleague parent sibling or therapist can help url url,pos
4,user how can i best support you right now,pos
...,...,...
21214,read about how a tiny robin show susie her mom be near after death url url,pos
21215,the bank have understand the error in their way and fully understand without truth and honesty at the bedrock and foundation of family we can never be secure until we reach retirement,pos
21216,things i can only say in url,pos
21217,things i can only say in url,pos


In [29]:
final_df

,tweet,label
0,globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression suffer with depression yourself check out url and have a look at both issue,pos
1,they just do it understand eggwhite medium star sparkleswhite question markheart with ribbon,pos
2,sometimes i be think to myself when some of you post depress shit like what the hell you be depress about the scenario you make in your head nothing ever happen to you loudly cry faceface with tear of joyface with tear of joy,pos
3,let u get talk this week mental health awareness week talk to a friend colleague parent sibling or therapist can help url url,pos
4,user how can i best support you right now,pos
...,...,...
21214,read about how a tiny robin show susie her mom be near after death url url,pos
21215,the bank have understand the error in their way and fully understand without truth and honesty at the bedrock and foundation of family we can never be secure until we reach retirement,pos
21216,things i can only say in url,pos
21217,things i can only say in url,pos


In [30]:
final_df.to_csv('final_data1-new.csv')

In [31]:
new_df = df.drop(['username', 'sentiment', 'text_lower', 'text_no_user', 'text_no_url', 'text_no_emoji', 'text_expanded', 'text_lemma', 'text_no_hash', 'text_tokenized', 'sentiment_processed', 'new_vader_sentiment', 'label'], axis=1)

In [32]:
new_df.head(50)

,text,processed_data
0,🌐 NEW ON THE WEBSITE 🌐\nThis week we are talking about #depression\nKnow anyone who suffers with depression?? Suffer with depression yourself??\nCheck out: https://t.co/M8qzkc4g4h and have a look at both issues :),globe with meridian new on the website globe with meridian this week we be talk about know anyone who suffer with depression suffer with depression yourself check out url and have a look at both issue
1,They just did it understand #depression 🫖🥚⭐️✨❔💝,they just do it understand eggwhite medium star sparkleswhite question markheart with ribbon
2,Sometimes I be thinking to myself when some of you post depressing shit like “what the hell you’re depressed about the scenarios you make in your head” nothing ever happens to you 😭😂😂 #bipolar #ADHD #depression #FamilyGuy #fun,sometimes i be think to myself when some of you post depress shit like what the hell you be depress about the scenario you make in your head nothing ever happen to you loudly cry faceface with tear of joyface with tear of joy
3,"Let’s get talking this week, Mental Health Awareness week. Talking to a friend, colleague, parent, sibling or therapist can help. https://t.co/ATM0h2mzrl. #mentalhealthawareness #talking #anxiety #depression #trauma #yourjourneystartshere #therapy #counsellor #help #surrey https://t.co/wh2yxkXS1S",let u get talk this week mental health awareness week talk to a friend colleague parent sibling or therapist can help url url
4,@Iamdepr47974144 How can I best support you right now? #mentalhealth #depression,user how can i best support you right now
5,Want To Come Out Of Depression ??? Then Follow The GivenTips To Over Come Out Of Depression...\n#depression #DepressionIsReal #getout #hope #lovelive #love #faith https://t.co/U8z9Htmr89,want to come out of depression then follow the giventips to over come out of depression url
6,#selfcare becomes so important with this. If you have the proper support then even better but if not remember to not be hard on yourself. You are not alone.\n#depression \n#DepressionIsReal \n#CPTSD \n#PTSD https://t.co/dZEAQSKKUA,become so important with this if you have the proper support then even well but if not remember to not be hard on yourself you be not alone url
7,"I can tell my low spell isn't going away because I just changed my entire Twitter profile and I still think it's garbage. This also isn't helping get homework graded, but it's okay, I still can't sleep so I have some extra time. #depression",i can tell my low spell be not go away because i just change my entire twitter profile and i still think it be garbage this also be not help get homework graded but it be okay i still cannot sleep so i have some extra time
8,"Go easy on yourself! According to research, striving for perfection can create #Stress #Depression #Anxiety and poor #MentalHealth And stop worrying about #Security &amp; #Safety issues as we can help and advise you on both @WatchtowerSecu1 @MatesInMind #MentalHealth #ItsOKNotToBeOK https://t.co/8hOEHp7DZw",go easy on yourself accord to research strive for perfection can create and poor and stop worry about and issue a we can help and advise you on both user user url
9,Talk to a psychotherapist online via Skype https://t.co/VrCrUyLv23\n\n#onlinetherapy #onlinetherapist #mentalhealth #anxiety #depression,talk to a psychotherapist online via skype url
